In [1]:
# bob.refresh {"cron": "10,35 19-21 * * *", "timeout": 40000}

In [2]:
%run __init__.ipynb

Successfully connected to MongoDB


In [3]:
#--- Covid-19 France data updated everyday at 19:00PM
if int(datetime.now().strftime('%H')) > 19:
    %run "covid-19_france.ipynb"

In [4]:
#--- Load data source
start_time = time.time()
db_all_fr = get_datasource('FRANCE_DB_ALL')
db_trend_fr = get_datasource('FRANCE_DB_TREND')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

Script execution completed at 15/04/2020 15:28:26. Time: --- 0.15306377410888672 secnds ---


In [15]:
#-- Covid-19 - DATA.GOUV.FR TENDANCES
start_time = time.time()
def domain_105(df,domain_num):
    #-- Create Datamodel
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'GROUPS': domain['STATUS_NAME'],
        'GROUPS_ORDER': domain['STATUS_ORDER'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'BOTTOM_FILTER': 'Depuis le début',
        'UPPER_FILTER_M': domain['METRIC'],
        'UPPER_FILTER_R': domain['LABEL'],
        'LABEL_GROUPS': domain['LABEL_GROUPS'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
    })
    
    #-- Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARV','UPPER_FILTER_M'] = 'VARIATION EN NB'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARP','UPPER_FILTER_M'] = 'VARIATION EN %'
    
    #-- Create new date scenario in BOTTOM_FILTER
    df1 = get_lastdays(domain, 15,"15 derniers jours",'BOTTOM_FILTER')
    domain = pd.concat([domain,df1],axis=0)
    
    #-- Filter upperfilter value to get France and detailed by region
    domain_dep = domain[domain['LABEL_GROUPS'] != 'France']
    domain_fra = domain[domain['UPPER_FILTER_R'] == 'France']
    domain = pd.concat([domain_dep,domain_fra],axis=0)
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    bob.mongo.save_df(domain,domain_num,db_app,True)
    
    #-- Create upper filter domain
    upper_filter = domain[['UPPER_FILTER_R']].drop_duplicates().sort_values(by='UPPER_FILTER_R', ascending=True).reset_index(drop=True)
    upper_filter['ORDER'] = upper_filter.index + 1
    upper_filter.loc[upper_filter['UPPER_FILTER_R'] == 'France', 'ORDER'] = 0
    upper_filter = upper_filter.sort_values(by='ORDER',ascending=True)
    upper_filter.to_csv(output_folder + '105_UPPER_FILTER.csv', sep=';')
    bob.mongo.save_df(upper_filter,'105_UPPER_FILTER',db_app,True)
    return domain

domain105 = domain_105(db_trend_fr,'105')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain105

Dataframe 105 successfully save in database app-wsr-dev in MongoDB. Time: --- 14.007291555404663 secnds ---
Dataframe 105_UPPER_FILTER successfully save in database app-wsr-dev in MongoDB. Time: --- 0.021067380905151367 secnds ---
Script execution completed at 13/04/2020 10:20:23. Time: --- 14.634268760681152 secnds ---


,ENTITY,SCENARIO,GROUPS,GROUPS_ORDER,DATE_ORDER,BOTTOM_FILTER,UPPER_FILTER_M,UPPER_FILTER_R,LABEL_GROUPS,VALUE,UNIT_VALUE,PRECISION,LAST_UPDATE
0,France,12/04/2020,Hospitalisations,1.0,20200412,Depuis le début,EVOLUTION,31 - Haute-Garonne,Occitanie,244.000000,cas,",.0f",13/04/2020 10:20:09
1,France,12/04/2020,Hospitalisations,1.0,20200412,Depuis le début,EVOLUTION,32 - Gers,Occitanie,28.000000,cas,",.0f",13/04/2020 10:20:09
2,France,12/04/2020,Hospitalisations,1.0,20200412,Depuis le début,EVOLUTION,33 - Gironde,Nouvelle-Aquitaine,299.000000,cas,",.0f",13/04/2020 10:20:09
3,France,12/04/2020,Hospitalisations,1.0,20200412,Depuis le début,EVOLUTION,34 - Hérault,Occitanie,183.000000,cas,",.0f",13/04/2020 10:20:09
4,France,12/04/2020,Hospitalisations,1.0,20200412,Depuis le début,EVOLUTION,35 - Ille-et-Vilaine,Bretagne,182.000000,cas,",.0f",13/04/2020 10:20:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48955,France,29/03/2020,Retours à domicile,3.0,20200329,15 derniers jours,VARIATION EN NB,France,France,507.000000,cas,"+,.0f",13/04/2020 10:20:09
48956,France,29/03/2020,Hospitalisations,1.0,20200329,15 derniers jours,VARIATION EN %,France,France,9.846416,%,"+,.2f",13/04/2020 10:20:09
48957,France,29/03/2020,Décès à l’hôpital,4.0,20200329,15 derniers jours,VARIATION EN %,France,France,12.618842,%,"+,.2f",13/04/2020 10:20:09
48958,France,29/03/2020,En réanimation,2.0,20200329,15 derniers jours,VARIATION EN %,France,France,8.404155,%,"+,.2f",13/04/2020 10:20:09


In [6]:
#-- Covid-19 - DATA.GOUV.FR CLASSEMENT
start_time = time.time()
def domain_107(df,domain_num):
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'CHILD': domain['LABEL'],
        'PARENT': domain['LABEL_GROUPS'],
        'UPPER_FILTER_M': domain['STATUS_NAME'],
        'ORDER':domain['STATUS_ORDER'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VARP': ' %',
        'PRECISION_VALUE': ',.0f',
        'PRECISION_VAR': '+,.0f',
        'PRECISION_VARP': '+,.2f',
    })
    
    domain.loc[domain['PARENT'] == 'France', 'PARENT'] = 'root'
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    bob.mongo.save_df(domain,domain_num,db_app,True)
    return domain

domain107 = domain_107(db_all_fr,'107')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain107

Dataframe 107 successfully save in database app-wsr-dev in MongoDB. Time: --- 9.946475267410278 secnds ---
Script execution completed at 15/04/2020 15:29:49. Time: --- 10.077900886535645 secnds ---


,ENTITY,SCENARIO,DATE_ORDER,CHILD,PARENT,UPPER_FILTER_M,ORDER,VALUE,VAR,VARP,UNIT_VARP,PRECISION_VALUE,PRECISION_VAR,PRECISION_VARP,LAST_UPDATE
0,France,14/04/2020,20200414,Provence-Alpes-Côte d'Azur,root,En réanimation,2.0,422,-15,-3.432494,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
1,France,14/04/2020,20200414,Provence-Alpes-Côte d'Azur,root,Hospitalisations,1.0,1894,35,1.882733,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
2,France,14/04/2020,20200414,31 - Haute-Garonne,Occitanie,Hospitalisations,1.0,243,0,0.000000,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
3,France,14/04/2020,20200414,32 - Gers,Occitanie,Hospitalisations,1.0,32,1,3.225806,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
4,France,14/04/2020,20200414,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,1.0,317,6,1.929260,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12955,France,19/03/2020,20200319,87 - Haute-Vienne,Nouvelle-Aquitaine,Retours à domicile,3.0,2,0,0.000000,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
12956,France,19/03/2020,20200319,80 - Somme,Hauts-de-France,Retours à domicile,3.0,12,3,33.333333,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
12957,France,19/03/2020,20200319,79 - Deux-Sèvres,Nouvelle-Aquitaine,Retours à domicile,3.0,3,1,50.000000,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
12958,France,19/03/2020,20200319,78 - Yvelines,Ile-de-France,Retours à domicile,3.0,7,2,40.000000,%,",.0f","+,.0f","+,.2f",15/04/2020 15:29:39
